##### Sessions: EV/EVSE

In [1]:
from scipy.optimize import fsolve
from pyomo.environ import *
import numpy as np
import numpy as np
import pandas as pd
from openpyxl import Workbook
from datetime import datetime
import os
import ntpath

np.random.seed(42)

m = 20  #  number of EVSEs
n = 20  #  initial number of time periods in the future (can bceome less)
r = .5  # factor of Enexis grid connection

# --------------------------------------------------------------------------
# The array's for all EV's & SE together and the resulting minimum.
# EV = np.random.choice([EV_MPI, EV_MPI / 2, 3.33, 3.33 / 2], size=(m, n))
# SE = np.random.choice([SE_MPO], size=(m, n))
# EVSE = np.minimum(EV, SE)


SE_MPO = 7  #              EVSE max Power output for each time period
EV_MPI = 7  #              EV max Power input for each time period
EX_MPO = m * r * SE_MPO  # enexis max Power output for each time period (constant)
CAP = 70
K = 0.075
# --------------------------------------------------------------------------

soc_l = 0.0; soc_h = 0.2
d_c_l = 0.4; d_c_h = 1.0
dur_l = 1.0; dur_h = 8.0
eng_l = 0.3; eng_h = 0.6
p_rnd = True # else linear

# --------------------------------------------------------------------------
print_solver_outcome = False
print_EVSE_power = False

print_session_max_charge = False
print_desired_charge = False
print_realistic_charge = False
print_remaining_charge = False

print_parking_time = False
print_energy_price = False

# tested with:
# solver = "ipopt" #  m 50 EVSE's, > 50 time periods
# solver = "mosek" #  m 50 EVSE's, > 50 time periods
# solver = "cplex"  # m 50 EVSE's, 18 time periods
solver = "glpk" #     m 50 EVSE's, > 50 time periods
# solver = "gurobi" # m xx EVSE's, 19 time periods - no license yet

alpha = 1.0  # EVSE efficiency
beta = 1.0  #  customer satisfaction
gamma = 1.0  # cost of energy

##### Charging Profile

In [2]:
def cv_pwr(t: float, pm: float, k: float) -> float:
    return pm * np.exp(-k * t)


def cv_eng(t2, t1, pm, k) -> float:
    return (-1 / k) * (cv_pwr(t2, pm, k) - cv_pwr(t1, pm, k))


def cv_pwr_avg(t2, t1, pm, k) -> float:
    return cv_eng(t2, t1, pm, k) / (t2 - t1)


def charge_profile(
    dur: float,
    soc: float,
    d_c: float,
    cap: float = CAP,
    ev_mpi: float = EV_MPI,
    k: float = K,
) -> list:
    """
    Calculate the charging profile for a given EV.

    Parameters
    ----------
    dur : float
        Duration of stay in hours.
    soc : float
        State of charge of the battery in %.
    d_c : float
        Desired charge of the battery in %.
    cap : float
        Capacity of the battery in kWh.
    ev_mpi : float
        Maximum power input of the EV in kW.
    k : float
        Charging curve constant.
        0.01-0.03 charge aggressively,
        0.05-0.1  prioritizing battery health and longevity
    cv : float
        Capacity at which the charging curve flattens in %.
    cc : float
        Capacity at which the charging curve starts in %.

    Returns
    -------
    list
        List of the charging profile in the format [time, power, charge].
    """
    # --------------------------------------------------------------------------
    # Calculate the charging profile
    # https://www.homechargingstations.com/ev-charging-time-calculator/
    # --------------------------------------------------------------------------
    # Calculate the charge required to reach the desired capacity
    cv = 0.8 * cap #                                 capacity at which the charging curve flattens [kWh]
    dc = d_c * cap #                                 soc + charge [kWh] required

    # current charge
    uc0 = soc * cap
    up0 = ev_mpi
    ut0 = uc0 / up0

    # first part of charge < 80% of cap
    uc1 = max(min(dc, cv) - min(uc0, cv), 0)  #      charge [kWh] required
    up1 = 0 if uc1 == 0 else ev_mpi  #               power [kW] required
    ut1 = 0 if uc1 == 0 else uc1 / up1  #            time [h] required
    # constrainted by duration
    ct1 = min(dur, ut1)  #                           real time [h]  stay
    cp1 = up1  #                                     real power [kW]  stay
    cc1 = ct1 * cp1  #                               charge [kWh] during stay

    # second part of charge > 80% of cap
    uc2 = max(max(dc, cv) - max(uc0, cv), 0)  #      charge [kWh] required
    up2 = 0  #                                       initial power [kW] required
    ut2 = 0  #                                       initial time [h] required
    ct2 = 0  #                                       real time [h]  stay
    cp2 = 0  #                                       real power [kW]  stay
    cc2 = 0  #                                       charge [kWh] during stay

    # Define the function for the given equation with specific ta, pm and k
    def zero_for_E(t2, t1, pm, k, E) -> float:
        return cv_eng(t2, t1, pm, k) - E

    # determime time and power for 2nd part of charge CV
    if uc2 > 0:
        # Solve the equation numerically
        # https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fsolve.html
        solution = fsolve(func=zero_for_E, x0=ut1, args=(0, ev_mpi, k, uc2), xtol=1e-3)

        ut2 = solution[0]  #                         time [h] required during CV
        up2 = cv_pwr(ut2, ev_mpi, k)  #              power [kW] required during CV

        ct2 = max(0, min(dur - ut1, ut2))  #         real time [h] during CV

        if ct2 > 0:
            cc2 = cv_eng(ct2, 0, ev_mpi, k)  #       charge [kWh] during CV
            cp2 = cv_pwr(ct2, ev_mpi, k)  #          real power [kW] during CV

    # --------------------------------------------------------------------------
    e2 = 0 if ut2 == 0 else float(zero_for_E(t2=ut2, t1=0, pm=ev_mpi, k=k, E=uc2))

    ut3 = max(0, dur - (ut1 + ut2))

    return {
        "params": {
            "dur": dur,
            "soc": soc,
            "d_c": d_c,
            "cap": cap,
            "ev_mpi": ev_mpi,
            "k": k,
        },
        "phase0": {"c0": uc0, "t": ut0, "p": up0},
        "phase1": {"c1": uc1, "t": ut1, "p": up1},
        "real_1": {"c1": cc1, "t": ct1 + ut3, "p": cp1},
        "phase2": {"c2": uc2, "t": ut1 + ut2, "p": up2},
        "real_2": {"c2": cc2, "t": ct1 + ct2 + ut3, "p": cp2},
        "result": {
            "ufc": uc0 + uc1 + uc2 + e2,
            "cfc": uc0 + cc1 + cc2 + e2,
            "dc": dc,
            "rm": dc - (uc0 + uc1 + uc2 + e2),
        },
        "tslots": {"t1": ut1, "T1": ct1, "t2": ut2, "T2": ct2, "t3": ut3, "T3": ut3},
    }

$$
\int_{a}^{b} P_{\text{max}} \times e^{-k \times (t - t_0)} \, dt = 
-\frac{P_{\text{max}}}{k} \left[ e^{-k \times (b - t_0)} - e^{-k \times (a - t_0)} \right]

$$


##### TGC: Central Controller

In [3]:
# read data from ev
cps = []
data = []
desired_charge = []
realistic_charge = []
state_of_charge = []
remaining_charge = []

for i in range(1, m + 1):
    cp = charge_profile(
        dur=np.random.uniform(dur_l, dur_h),
        soc=np.random.uniform(soc_l, soc_h),
        d_c=np.random.uniform(d_c_l, d_c_h),
        cap=CAP,
        ev_mpi=EV_MPI,
        k=K,
    )
    cps.append(cp)
    ev = "ev" + str(i).zfill(2)
    data.append({"ev": ev, "t": cp["real_1"]["t"], "p": cp["real_1"]["p"]})
    data.append({"ev": ev, "t": cp["real_2"]["t"], "p": cp["real_2"]["p"]})
    # list for quick lookups
    state_of_charge.append(cp["phase0"]["c0"])
    desired_charge.append(cp["result"]["dc"])
    realistic_charge.append(cp["result"]["cfc"])
    remaining_charge.append(cp["result"]["cfc"] - cp["phase0"]["c0"])
    
    # data.append({"ev": ev, "t": cp["real_3"]["t"], "p": cp["real_3"]["p"]})
    # print(ev, cp)
    # if i==1:
    #     print(cp["params"])
    #     print(cp["phase0"])
    #     print(cp["phase1"])
    #     print(cp["real_1"])
    #     print(cp["phase2"])
    #     print(cp["real_2"])
    #     print(cp["result"])
    #     print(cp["tslots"])


# Create a DataFrame from the data
df = pd.DataFrame(data)

# Remove all records where 't' is 0
df = df.loc[df["t"] != 0]

# df_pivot = df.pivot(index='ev', columns='t', values='p')
df_pivot = df.pivot_table(index="ev", columns="t", values="p", aggfunc="sum")

# Replace NaN values in the last column with 0 or ev_mpi
df_pivot.iloc[:, -1] = df_pivot.iloc[:, -1].fillna(0)  # ev_mpi


# Replace NaN values in the other columns with the last non-NaN value
def fill_na_with_last_val(row):
    last_val = None
    for col in reversed(row.index):
        if pd.isna(row[col]):
            if last_val is not None:
                row[col] = last_val
        else:
            last_val = row[col]
    return row


df_pivot = df_pivot.apply(fill_na_with_last_val, axis=1)

# Keep only the first X columns or fewer if the DataFrame has fewer than X columns
df_pivot = df_pivot.iloc[:, : min(n, df_pivot.shape[1])]

# ------------------------------------------------------------------------------
# print("\n")
# print(df_pivot)

# Convert the DataFrame to a NumPy array
EVSE = df_pivot.values

if print_EVSE_power:
    print(f"\nEVSE:\n {EVSE}\n")


In [4]:
# Create the weights w[j] for the parking
absolute_times = df_pivot.columns.tolist()

pt = [j - i for i, j in zip(absolute_times[:-1], absolute_times[1:])]
pt.insert(0, absolute_times[0])

# pt = np.random.choice([40, 10, 20, 10, 10], size=(n))/60
if print_parking_time:
    print(f"\nparking time:\n {pt}\n")
w = pt / np.sum(pt)

# print(sum(pt[0:])) 




In [5]:
n = len(pt) # number of time periods can become less when they overlap
# print(len(pt))

In [6]:
# desired charge for each session, max battery capacity = set to 70 kWh
desired_charge = np.array(desired_charge)

if print_desired_charge:
    print(f"\ndesired_charge:\n {desired_charge}\n")


In [7]:
# realistic charge for each session due to duration of stay
realistic_charge = np.array(realistic_charge)

if print_realistic_charge:
    print(f"\nrealistic_charge:\n {realistic_charge}\n")

In [8]:
# remaining charge for each session due to duration of stay
remaining_charge = np.array(remaining_charge)

if print_remaining_charge:
    print(f"\nremaining_charge:\n {remaining_charge}\n")

In [9]:
# energy price in euro/kWh for each time period
# EnergyPrice = np.array([0.04, 0.08, 0.31, 0.28, 0.12])

if p_rnd:
    EnergyPrice = np.random.uniform(eng_l, eng_h, n)
else:
    EnergyPrice = np.linspace(eng_l, eng_h, n)

Energy_Cost = EnergyPrice * pt

if print_energy_price:
    print(f"\nenergy_price:\n {EnergyPrice}\n")

##### OLP - algorithm

In [10]:
# --------------------------------------------------------------------------
# TGC: Tetris Game Charger
# --------------------------------------------------------------------------

# --------------------------------------------------------------------------
# Abstract Model
# https://pyomo.readthedocs.io/en/stable/pyomo_overview/simple_examples.html#a-simple-abstract-pyomo-model
# --------------------------------------------------------------------------

model = AbstractModel()

# --------------------------------------------------------------------------
# Sets
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Sets.html
# --------------------------------------------------------------------------

model.I = RangeSet(1, m)  # set of EVSEs
model.J = RangeSet(1, n)  # set of time periods for a certain horizon (h=5)


# --------------------------------------------------------------------------
# Parameters
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Parameters.html
# --------------------------------------------------------------------------

# energy price per kWh for each time period
energy_price = {(j + 1): EnergyPrice[j] for j in range(len(EnergyPrice))}
model.energy_price = Param(model.J, initialize=energy_price)

# enexis max Power output for each time period (constant)
enexis_kw_dv = {}  # deviations from the default value for enexis mpo per time period j
model.enexis = Param(model.J, initialize=enexis_kw_dv, default=EX_MPO)

# max Power EVSE session for each time period
session_max_kw = {
    (i + 1, j + 1): EVSE[i][j] for i in range(len(EVSE)) for j in range(len(EVSE[i]))
}
model.session = Param(model.I, model.J, initialize=session_max_kw)

# remaining charge for each session
remaining_charge_kwh = {
    (i + 1): remaining_charge[i] for i in range(len(remaining_charge))
}
model.remaining_charge = Param(model.I, initialize=remaining_charge_kwh)


# --------------------------------------------------------------------------
# Variables
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Variables.html
# --------------------------------------------------------------------------

model.x = Var(model.I, model.J, domain=NonNegativeReals)


# --------------------------------------------------------------------------
# Objective function
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Objectives.html
# --------------------------------------------------------------------------


def obj_expression(model):
    # return the expression for the objective
    return (
        alpha
        * sum(
            w[j - 1] * (1 - sum(model.x[i, j] for i in model.I) / model.enexis[j])
            for j in model.J
        )
        + beta
        * sum(
            (
                1
                - sum(pt[j - 1] * model.x[i, j] for j in model.J)
                / model.remaining_charge[i]
            )
            for i in model.I
        )
        / m
        + (gamma / sum(Energy_Cost[j - 1] * model.enexis[j] for j in model.J))
        * sum(
            (Energy_Cost[j - 1] * sum(model.x[i, j] for i in model.I)) for j in model.J
        )
    )


model.OBJ = Objective(rule=obj_expression)

# --------------------------------------------------------------------------
# Constraints
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Constraints.html
# https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Expressions.html
# --------------------------------------------------------------------------


# is ev/se power[i] <= enexis max power output for period j?
def ex_grid_constraint_rule(model, j):
    return sum(model.x[i, j] for i in model.I) <= model.enexis[j]


# is ev/se power <= session max power input for period j?
def session_constraint_rule(model, i, j):
    return model.x[i, j] <= model.session[i, j]


# is final charge <= realistic charge <= desired charge
def deschrg_constraint_rule(model, i):
    return sum(pt[j - 1] * model.x[i, j] for j in model.J) <= model.remaining_charge[i]


# the next line creates one constraint for each member of the set model.J
model.Ex_Grid_Constraint = Constraint(model.J, rule=ex_grid_constraint_rule)
model.DesChrg_Constraint = Constraint(model.I, rule=deschrg_constraint_rule)
model.Session_Constraint = Constraint(model.I, model.J, rule=session_constraint_rule)

# --------------------------------------------------------------------------
# create a model instance and optimize
# https://pyomo.readthedocs.io/en/stable/working_abstractmodels/instantiating_models.html
# https://link.springer.com/book/10.1007/978-3-030-68928-5
# --------------------------------------------------------------------------

tgc = model.create_instance()

opt = pyomo.environ.SolverFactory(solver)

opt.solve(tgc)

# --------------------------------------------------------------------------
# display solution
# --------------------------------------------------------------------------

# outcome of solver model
if print_solver_outcome:
    print(f"\nsolver outcome:\n {tgc.pprint()}\n")




### Result DataFrames

In [11]:
# function(s) to create a DataFrame with the results
def create_df_evse_cs_results():
    tmp = []
    for i in range(1, m + 1):
        tmp.append(
            {
                "EVSE": "EVSE" + str(i).zfill(2),
                "STAY": round(cps[i - 1]["params"]["dur"], 2),
                "SC%": round(100 * state_of_charge[i - 1] / CAP, 2),
                "DC%": round(100 * desired_charge[i - 1] / CAP, 2),
                "DC": round(desired_charge[i - 1], 2),
                "RC": round(realistic_charge[i - 1], 2),
                "FC": round(
                    sum(pt[x - 1] * tgc.x[i, x].value for x in range(1, n + 1))
                    + state_of_charge[i - 1],
                    2,
                ),
                "CS": round(
                    100
                    * (
                        sum(pt[x - 1] * tgc.x[i, x].value for x in range(1, n + 1))
                        + state_of_charge[i - 1]
                    )
                    / realistic_charge[i - 1],
                    2,
                ),
                "ENG0": state_of_charge[i - 1],
            }
        )
    return pd.DataFrame(tmp)


def create_df_evse_charge_results():
    tmp = []
    for i in range(1, m + 1):
        for x in range(1, n + 1):
            tmp.append(
                {
                    "EVSE": "EVSE" + str(i).zfill(2),
                    "TIME": "T"
                    + str(x).zfill(2)
                    + "_"
                    + str(round(pt[x - 1], 2)).ljust(4, "0"),
                    "POWR": tgc.x[i, x].value,
                    "ENG+": pt[x - 1] * tgc.x[i, x].value,
                    "COST": EnergyPrice[x - 1] * pt[x - 1] * tgc.x[i, x].value,
                }
            )
    return pd.DataFrame(tmp)


df_evse_cs_results = create_df_evse_cs_results()
# print(f"\nEVSE_CS:\n{df_evse_cs_results}")

# Overall most detailed results in long format
df_evse_charge_results = create_df_evse_charge_results()
# print(f"\nEVSE_charge:\n{df_evse_charge_results}")

# Overall most detailed results in wide format
df_evse_charge_results_melted = df_evse_charge_results.melt(
    id_vars=["EVSE", "TIME"],
    value_vars=["POWR", "ENG+", "COST"],
    var_name="TYPE",
    value_name="VALUE",
)

dfMTRX = df_evse_charge_results_melted.pivot(
    index=["EVSE", "TYPE"], columns="TIME", values="VALUE"
).reset_index()

# Aggregated results per EVSE
df_evse_charge_results_tot = (
    df_evse_charge_results_melted.groupby(["EVSE", "TYPE"])["VALUE"]
    .sum()
    .reset_index()
    .pivot(index="EVSE", columns="TYPE", values="VALUE")
    .reset_index()
)

# print(f"\nEVSE_charge_tot:\n{df_evse_charge_results_tot}")

# DataFrame formatting
dfEVSE = df_evse_cs_results.merge(df_evse_charge_results_tot, on="EVSE", how="inner")
dfEVSE.drop(columns=["POWR"], inplace=True)
dfEVSE["ENG0"] = dfEVSE["ENG0"].apply(lambda x: round(x, 2))
dfEVSE["ENG+"] = dfEVSE["ENG+"].apply(lambda x: round(x, 2))
dfEVSE["COST"] = dfEVSE["COST"].apply(lambda x: round(x, 2))
dfEVSE = dfEVSE.reindex(
    columns=[
        "EVSE",
        "STAY",
        "SC%",
        "DC%",
        "DC",
        "RC",
        "FC",
        "CS",
        "ENG0",
        "ENG+",
        "COST",
    ]
)

# Aggregated results per TIME Slot
df_time_charge_results_tot = (
    df_evse_charge_results_melted.groupby(["TIME", "TYPE"])["VALUE"]
    .sum()
    .reset_index()
    .pivot(index="TIME", columns="TYPE", values="VALUE")
    .reset_index()
)

# DataFrame formatting
dfTIME = df_time_charge_results_tot
dfTIME["PRICE"] = EnergyPrice.round(2)
dfTIME["POWR"] = dfTIME["POWR"].apply(lambda x: round(x, 2))
dfTIME["ENG+"] = dfTIME["ENG+"].apply(lambda x: round(x, 2))
dfTIME["COST"] = dfTIME["COST"].apply(lambda x: round(x, 2))
dfTIME["PWR%"] = dfTIME["POWR"].apply(lambda x: round(100 * x / EX_MPO, 2))
dfTIME = dfTIME.reindex(
    columns=[
        "TIME",
        "PRICE",
        "COST",
        "ENG+",
        "POWR",
        "PWR%",
    ]
)

# print(f"\nTIME_charge_tot:\n{df_time_charge_results_tot}")

## Print

In [12]:
# print the objective outcome
print(f"\nObjective Outcome = {round(value(tgc.OBJ), 4)}")

print(f"\nEVSE Results\n\n{dfEVSE}\n")
# print(f"\nEVSE Total\nCOST: {round(dfEVSE['COST'].sum(),2)}\nENG+: {round(dfEVSE['ENG+'].sum(),2)}")

print(f"\nTIME Results\n\n{dfTIME}\n")
# print(f"\nTIME Total\nCOST: {round(dfTIME['COST'].sum(),2)}\nENG+: {round(dfTIME['ENG+'].sum(),2)}")

print(
    f"""\n \
    Totals\n\n \
    TIME: {round(sum(pt),2)} hrs\n \
    COST: € {round(dfTIME['COST'].sum(),2)}\n \
    ENG+: {round(dfTIME['ENG+'].sum(),2)} kWh\n \
    CSAT: {round(dfEVSE['CS'].mean(), 2)} %\n """
)


Objective Outcome = 1.1794

EVSE Results

      EVSE  STAY    SC%    DC%     DC     RC     FC      CS   ENG0   ENG+  \
0   EVSE01  3.62  19.01  83.92  58.74  38.66  38.66  100.00  13.31  25.35   
1   EVSE02  5.19   3.12  49.36  34.55  34.55  29.07   84.14   2.18  26.89   
2   EVSE03  1.41  17.32  76.07  53.25  21.97  21.97  100.00  12.13   9.85   
3   EVSE04  5.96   0.41  98.19  68.74  41.98  13.82   32.91   0.29  13.53   
4   EVSE05  6.83   4.25  50.91  35.64  35.64  34.33   96.33   2.97  31.36   
5   EVSE06  2.28   6.08  71.49  50.04  20.25  20.25  100.00   4.26  15.99   
6   EVSE07  4.02   5.82  76.71  53.70  32.24  32.24  100.00   4.08  28.17   
7   EVSE08  1.98   5.84  61.98  43.39  17.93  17.93  100.00   4.09  13.84   
8   EVSE09  4.19  15.70  51.98  36.39  36.39  36.39  100.00  10.99  25.39   
9   EVSE10  4.60  11.85  42.79  29.95  29.95  29.95  100.00   8.29  21.66   
10  EVSE11  5.25   3.41  43.90  30.73  30.73  30.73  100.00   2.39  28.34   
11  EVSE12  7.64  19.31  88.50  6

## Export Results

In [13]:
def unix_to_windows_path(unix_path: str) -> str:
    """
    Convert a unix path to a windows path.
    Parameters
    ----------
    unix_path : str
        The unix path.
    Returns
    -------
    str
        The windows path.
    """
    windows_path = ntpath.join(*unix_path.split(os.sep)[2:])
    windows_path = windows_path[0] + ":" + windows_path[1:]

    return windows_path


def create_full_filename(file_name) -> str:
    """
    Create a full filename with timestamp and path.

    Parameters
    ----------
    file_name : str
        The name of the file.

    Returns
    -------
    str
        The full filename with timestamp and path.
    """

    FLD = "/mnt/c/JADS/TGC/Results"
    if not os.path.exists(FLD):
        os.makedirs(FLD)

    FND = datetime.now().strftime("%Y%m%d_%H%M%S")
    XLS = ".xlsx"

    return os.path.join(FLD, FND + "_" + file_name + XLS)

# Write results to Excel for MTRX
FFN = create_full_filename("dfMTRX")
dfMTRX.to_excel(FFN, sheet_name="MTRX", index=False)
os.system('cmd.exe /C start excel.exe "' + unix_to_windows_path(FFN) + '"')

# Write results to Excel for EVSE
FFN = create_full_filename("dfEVSE")
dfEVSE.to_excel(FFN, sheet_name="EVSE", index=False)
os.system('cmd.exe /C start excel.exe "' + unix_to_windows_path(FFN) + '"')

# Write results to Excel for TIME
FFN = create_full_filename("dfTIME")
dfTIME.to_excel(FFN, sheet_name="TIME", index=False)
os.system('cmd.exe /C start excel.exe "' + unix_to_windows_path(FFN) + '"')

'\\wsl.localhost\Ubuntu-22.04\home\floris\sim\sal_jads\floris_tetris\olp'
CMD.EXE was started with the above path as the current directory.
UNC paths are not supported.  Defaulting to Windows directory.
'\\wsl.localhost\Ubuntu-22.04\home\floris\sim\sal_jads\floris_tetris\olp'
CMD.EXE was started with the above path as the current directory.
UNC paths are not supported.  Defaulting to Windows directory.
'\\wsl.localhost\Ubuntu-22.04\home\floris\sim\sal_jads\floris_tetris\olp'
CMD.EXE was started with the above path as the current directory.
UNC paths are not supported.  Defaulting to Windows directory.


0

In [14]:
unix_path = "/mnt/c/JADS/TGC/Results"
windows_path = ntpath.join(*unix_path.split(os.sep)[2:])

print(windows_path)

c\JADS\TGC\Results
